In [1]:
import random
import string
import re
import textwrap
import numpy as np

In [2]:
## create substitution cipher

letters1 = list(string.ascii_lowercase)
letters2 = list(string.ascii_lowercase)

# shuffle the second set of letters
random.shuffle(letters2)

# populate map
true_mapping = {}
for k, v in zip(letters1, letters2):
    true_mapping[k] = v

In [3]:
regex = r"[^a-zA-Z]"
def encode_message(msg):
    """A function to encode a message"""
    msg = msg.lower()
    msg = re.sub(regex, " ", msg)
    coded_msg = []
    for ch in msg:
        coded_ch = ch  # could just be a space
        if ch in true_mapping:
            coded_ch = true_mapping[ch]
        coded_msg.append(coded_ch)
    return "".join(coded_msg)

In [4]:
def decode_message(msg, word_map):
    """A function to decode a message"""
    decoded_msg = []
    for ch in msg:
        decoded_ch = ch  #could just be a space
        if ch in word_map:
            decoded_ch = word_map[ch]
        decoded_msg.append(decoded_ch)
    return "".join(decoded_msg)

In [5]:
original_message = '''I then lounged down the street and found,
as I expected, that there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies I was compelled to listen to.
'''

### Test the encode and decode functions

In [6]:
encoded_message = encode_message(original_message)
encoded_message

'z lgbv jxcvibe expv lgb hlnbbl tve yxcve  th z bsqbklbe  lgtl lgbnb pth t abph zv t jtvb pgzkg ncvh expv dw xvb ptjj xy lgb itnebv  z jbvl lgb xhljbnh t gtve zv ncddzvi expv lgbzn gxnhbh  tve nbkbzube zv bskgtvib lpxqbvkb  t ijthh xy gtjy tve gtjy  lpx yzjjh xy hgti lxdtkkx  tve th ackg zvyxnatlzxv th z kxcje ebhznb tdxcl azhh tejbn  lx htw vxlgzvi xy gtjy t exfbv xlgbn qbxqjb zv lgb vbzigdxcngxxe zv pgxa z pth vxl zv lgb jbthl zvlbnbhlbe  dcl pgxhb dzxintqgzbh z pth kxaqbjjbe lx jzhlbv lx  '

In [7]:
word_map = {v:k for k, v in true_mapping.items()}
decode_message(encoded_message, word_map)

'i then lounged down the street and found  as i expected  that there was a mews in a lane which runs down by one wall of the garden  i lent the ostlers a hand in rubbing down their horses  and received in exchange twopence  a glass of half and half  two fills of shag tobacco  and as much information as i could desire about miss adler  to say nothing of half a dozen other people in the neighbourhood in whom i was not in the least interested  but whose biographies i was compelled to listen to  '

## Create a language model
p(x1, x2, x3, ..., xT) = p(x1) * p(x2|x1) * p(x3|x2) * ...  
- first letter -> unigram
- other letters -> bigram

In [8]:
# initial state distribution
pi = np.zeros(26)

def update_pi(ch):
    """A function to update the initial state distribution"""
    i = ord(ch) - 97  # ord("a") = 97
    pi[i] += 1

In [9]:
# initialize Markov matrix
M = np.ones((26, 26))

def update_transition(ch1, ch2):
    """A function to update the Markov matrix"""
    i = ord(ch1) - 97
    j = ord(ch2) - 97
    M[i,j] += 1

In [10]:
# for replacing non-alpha characters
regex = r"[^a-zA-Z]"
f = open("moby_dick.txt")
for line in f:
    line = line.rstrip()

    # there are blank lines in the file
    if line:
        line = re.sub(regex, " ", line)

    # split the tokens in the line and lowercase
    tokens = line.lower().split()
    for token in tokens:
        # update the model

        # first letter
        ch0 = token[0]
        update_pi(ch0)

        # other letters
        for ch1 in token[1:]:
            update_transition(ch0, ch1)
            ch0 = ch1


# normaliza the probabilities
pi /= pi.sum()
M /= M.sum(axis=1, keepdims=True)

f.close()

In [11]:
def get_word_prob(word):
    """get the log-probability of a word / token"""
    i = ord(word[0]) - 97
    logp = np.log(pi[i])

    for ch in word[1:]:
        j = ord(ch) - 97
        logp += np.log(M[i, j])  #update prob
        i = j #update j

    return logp

In [12]:
def get_sequence_prob(words):
    """get the probability of a sequence of words"""
    if type(words) == str:
        words = words.lower().split()

    logp = 0
    for word in words:
        logp += get_word_prob(word)
    return logp

## Run evolutionary algorithm to decode the message

In [13]:
def create_dna_pool():
    dna_pool = []
    for _ in range(20):
        dna = list(string.ascii_lowercase)
        random.shuffle(dna)
        dna_pool.append(dna)
    return dna_pool

In [14]:
def evolve_offspring(dna_pool, n_children):
    offspring = []
    for dna in dna_pool:
        for _ in range(n_children):
            copy = dna.copy()
            j = np.random.randint(len(copy))
            k = np.random.randint(len(copy))

            # swich 2 letters in copy of dna
            tmp = copy[j]
            copy[j] = copy[k]
            copy[k] = tmp
            offspring.append(copy)
    return offspring + dna_pool

In [15]:
num_iters = 1000
scores = np.zeros(num_iters)
best_dna = None
best_map = None
best_score = float("-inf")
dna_pool = create_dna_pool()

for i in range(num_iters):
    if i > 0:
        # get offspring from the current dna pool
        dna_pool = evolve_offspring(dna_pool, 3)

    # calculate score for each dna
    dna2score = {}
    for dna in dna_pool:
        current_map = {}
        for k, v in zip(letters1, dna):
            current_map[k] = v
        decoded_message = decode_message(encoded_message, current_map)
        score = get_sequence_prob(decoded_message)

        # needs to be a string to be a dict key
        dna2score["".join(dna)] = score
        if score > best_score:
            best_dna = dna
            best_map = current_map
            best_score = score

    scores[i] = np.mean(list(dna2score.values()))

    # keep the best five dna
    # also turn them back into list of single chars
    sorted_dna = sorted(dna2score.items(), key=lambda x: x[1], reverse=True)
    dna_pool = [list(k) for k, _ in sorted_dna[:5]]

    if i % 200 == 0:
        print(f"iter: {i}, score: {scores[i]}, best so far: {best_score}")

iter: 0, score: -2106.903071214343, best so far: -1858.5518885747967
iter: 200, score: -1095.9411836780946, best so far: -951.2496074296321
iter: 400, score: -993.6622749482083, best so far: -929.5902922650557
iter: 600, score: -1003.0990856838407, best so far: -929.5902922650557
iter: 800, score: -1001.039172957757, best so far: -929.5902922650557


In [16]:
# use best score
decoded_message = decode_message(encoded_message, best_map)

In [17]:
print(f"LL of decoded message: {get_sequence_prob(decoded_message)}")
print(f"LL of true message: { get_sequence_prob(re.sub(regex, ' ', original_message.lower()))}")

LL of decoded message: -929.5902922650557
LL of true message: -933.0312453751817


In [18]:
# which letters are wrong?
num_wrongs = 0
for true, v in true_mapping.items():
    pred = best_map[v]
    if true != pred:
        print(f"true: {true}, pred: {pred}")
        num_wrongs += 1
accuracy = 100 * (26 - num_wrongs) / 26
print(f"accuracy: {accuracy:.2f}%")


true: j, pred: q
true: k, pred: j
true: q, pred: z
true: z, pred: k
accuracy: 84.62%


In [19]:
# print the final decoded message
print(f"Decoded message: \n{textwrap.fill(decoded_message)}")
print(f"\nTrue message:\n{original_message}")

Decoded message: 
i then lounged down the street and found  as i expected  that there
was a mews in a lane which runs down by one wall of the garden  i lent
the ostlers a hand in rubbing down their horses  and received in
exchange twopence  a glass of half and half  two fills of shag tobacco
and as much information as i could desire about miss adler  to say
nothing of half a doken other people in the neighbourhood in whom i
was not in the least interested  but whose biographies i was compelled
to listen to

True message:
I then lounged down the street and found,
as I expected, that there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies I was c